In [1]:
from datetime import date, timedelta
import os
import dill as pickle
import sys
import pandas as pd
import numpy as np
sys.path.append("..")

from abautomator.visualizer import BasicVisualizer, AbsDiffVisualizer, RelDiffVisualizer, utils
from abautomator import config, describer, analyzer

from bokeh.io import output_notebook
from bokeh.plotting import show, save
# from bokeh.models import MultiChoice, CustomJS, Button
from bokeh.layouts import column, row

output_notebook()
pd.set_option('display.max_colwidth', None)

Loading BokehJS ...

In [2]:
# IMPORTANT! Make sure these global variables are set
from abautomator.exp_config import EXP_NAME, CTRL_NAME, COND_ORDER
from abautomator.exp_config import PRIMARY_METRIC_LIST, SECONDARY_METRIC_LIST, GUARDRAIL_METRIC_LIST

In [3]:
def filter_coll_metrics(coll, column, value):
    for metric in coll.metrics:
        metric.user_metric_df = metric.user_metric_df[metric.user_metric_df[column] == value]
    
    return coll

In [4]:
local_coll = pickle.load(
    open(os.path.join("..", "tests", "cache", f"{EXP_NAME}_coll.p"), "rb" )
)

In [8]:
import copy

for mapping in [
        (None, "all"),
        ("device_type","android"),
        ("device_type", "ios"),
        ("user_type","new"),
        ("user_type", "existing"),
    ]:
    curr_coll = copy.deepcopy(local_coll)
    column, value = mapping
    if value != "all":
        column, value = mapping
        curr_coll = filter_coll_metrics(curr_coll, column=column, value=value)

    print(f"describing data for {value}")
    desc = describer.Describer(
        metrics=curr_coll.metrics
    )
    outcomes_dict = desc.describe_data(exp_name=EXP_NAME)
    
    print(f"analyzing data for {value}")
    analy =  analyzer.Analyzer(
        outcomes=outcomes_dict,
        ctrl_name=CTRL_NAME,
    )

    pickle.dump(
        analy, open(f"{EXP_NAME}_analy_{value}.p", "wb" )
    )

describing data for all
analyzing data for all
describing data for android
analyzing data for android
describing data for ios
analyzing data for ios
describing data for new
analyzing data for new
describing data for existing
analyzing data for existing


In [6]:
def process_metric_set(metric_list, metric_list_label):
    for mapping in [
                (None, "all"),
                ("device_type","android"),
                ("device_type", "ios"),
                ("user_type","new"),
                ("user_type", "existing"),
            ]:
        _, value = mapping

        local_analy = pickle.load(
            open(os.path.join("..", "tests", "cache", f"{EXP_NAME}_analy_{value}.p"), "rb" )
        )
        output_df = local_analy.get_rel_diff_confidence_intervals()
        df = utils.order_categories(output_df, metric_list, COND_ORDER)
        df = utils.remove_categories_w_no_order(df)
        source = utils.convert_df_to_source(df)

        vis = RelDiffVisualizer(source, title=f"{metric_list_label.title()} - {value.title()}")
        fig = vis.get_figure()
        to_show = vis.get_layout()
#         show(to_show)

        save(to_show, f"/home/jovyan/work/notebooks/{value}-{metric_list_label.replace(' ', '-')}.html")

In [7]:
process_metric_set(PRIMARY_METRIC_LIST, "primary metrics")

NaN values found!
/opt/conda/lib/python3.9/site-packages/bokeh/io/saving.py:126: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/opt/conda/lib/python3.9/site-packages/bokeh/io/saving.py:139: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")
NaN values found!
NaN values found!
NaN values found!
NaN values found!


In [8]:
process_metric_set(SECONDARY_METRIC_LIST, "secondary metrics")

NaN values found!
NaN values found!
NaN values found!
NaN values found!
NaN values found!


In [9]:
process_metric_set(GUARDRAIL_METRIC_LIST, "guardrail metrics")

NaN values found!
NaN values found!
NaN values found!
NaN values found!
NaN values found!
